# Busca Cega

Abordagem de busca que não considera nenhuma informação sobre qual sucessor é mais promissor para atingir uma meta (objetivo)
###### Passos
- Formular objetivo
- Formular a busca
- Executar

### Modelando o problema
- Sliding Puzzle
<img src="images/sliding_puzzle.gif" width="250" align="center">

In [21]:
import numpy as np

class SlidingPuzzle():
    def __init__(self, num_blocos):
        '''
        Construtor
        Args:
            - num_blocos: numero de blocos por linha e coluna, valor inteiro (Ex: 3 significa 3 linhas e 3 colunas)
        '''
        self.num_blocos = num_blocos

    def _encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está
        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if estado[i, j] == elemento:
                    return i, j
        return None, None

    def verifica_estados(self, atual, objetivo):
        '''
        Verifica se o estado atual é o objetivo da busca
        Args:
            - atual: matriz que descreve o estado atual
            - objetivo: matriz que descreve o estado objetivo
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        flag = True
        for i in range(self.num_blocos):
            for j in range(self.num_blocos):
                if atual[i, j] != objetivo[i, j]:
                    flag = False
                    break

        return flag

    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados
        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        
        novos_estados = []
        linha, coluna = self._encontra_posicao(atual, 0)

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            bloco_alvo = novo_estado[nova_linha, coluna]
            novo_estado[nova_linha, coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.num_blocos - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            bloco_alvo = novo_estado[linha, nova_coluna]
            novo_estado[linha, nova_coluna] = 0
            novo_estado[linha, coluna] = bloco_alvo

            novos_estados.append(novo_estado)

        return novos_estados


### Formulando Busca

#### Busca em largura (BrFS– Breadth-first search)

Realiza a busca em nível. Imagine uma árvore de estados, nela a busca é realizada sequencialmente em cada nó do mesmo nível

<img src="images/bfs.gif" width="250" align="center">

In [22]:
from queue import Queue
# Classe 
class BreadthFirstSearch():
    
    def __init__(self, problema):
        self.problema = problema
        
    def _verifica_visitado(self, estado, estados_visitados):
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False
        
    def busca(self, inicio, fim):
        fila = Queue()
        fila.put(inicio)
        
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        
        while not fila.empty():
            atual = fila.get()
            estados_visitados.append(atual)
            
            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
                
            else:
                cont_estados += 1
                print(f"Visitando #{cont_estados}")
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        fila.put(i)
                        
        return solucao_encontrada, estados_visitados, cont_estados
            
## Construtor 
## Verificar se estado já foi visitado
## Busca
## Enquanto houver elementos na fila
### Verificar objetivo
### Expandir estados
### Expandir estados



#### Busca em profundidade (DFS – Depth-first search)

Realiza a busca por ramo. Imagine uma árvore de estados, nela a busca é realizada sequencialmente em cada ramo, e só após completá-lo, busca no ramo vizinho.

<img src="images/dfs.gif" width="250" align="center">

In [23]:
from queue import LifoQueue

class DepthFirstSearch():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def _verifica_visitado(self, estado, estados_visitados):
        '''
        Verifica se 'estado' está na lista de estados visitados
        Args:
            - estado: estado qualquer do tabuleiro
            - estados_visitados: lista com todos os estados já visitados
        Return:
            - booleano dizendo se o estado foi visitado ou não
        '''
        for i in estados_visitados:
            if self.problema.verifica_estados(i, estado):
                return True
        return False
    
    def busca(self, inicio, fim):
        '''
        Realiza a busca DFS, armazenando os estados em uma PILHA
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        pilha = LifoQueue()
        pilha.put(inicio)
        
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        
        while not pilha.empty():
            atual = pilha.get()
            estados_visitados.append(atual)
            
            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
                
            else:
                cont_estados += 1
                if cont_estados == 2000:
                    break
                else:
                    print(f"Visitando #{cont_estados}")
                    novos_estados = self.problema.expande_estados(atual)
                    for i in novos_estados:
                        if not self._verifica_visitado(i, estados_visitados):
                            pilha.put(i)
                        
        return solucao_encontrada, estados_visitados, cont_estados

### Executando

In [24]:
# Pacote auxiliar para o cálculo do tempo
from time import time

# Criando objeto do problema
problema = SlidingPuzzle(3)

# Criando Matriz inicial e matriz alvo
start = np.matrix([[1,0,2],[8,4,3],[7,6,5]])
target = np.matrix([[1,2,3],[8,0,4],[7,6,5]])

# Mostrando informações iniciais
print(f"Initial state: \n{start}")
print("*"*15)
print(f"Target state: \n{target}")
print("*"*15)

Initial state: 
[[1 0 2]
 [8 4 3]
 [7 6 5]]
***************
Target state: 
[[1 2 3]
 [8 0 4]
 [7 6 5]]
***************


In [25]:
# Execução do BFS
bfs = BreadthFirstSearch(problema)

ini = time() # Tempo inicial

bfs_solucao, bfs_estados_visitados, bfs_num_visitados = bfs.busca(start, target) # chamando busca

bfs_time = time()-ini # Tempo total

if bfs_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Visitando #1
Visitando #2
Visitando #3
Visitando #4
Visitando #5
Visitando #6
Visitando #7
Visitando #8
Visitando #9
Visitando #10
Visitando #11
Visitando #12
Visitando #13
Visitando #14
Visitando #15
Visitando #16
Visitando #17
Visitando #18
Solution found!!!


In [26]:
# Execução do DFS
dfs = DepthFirstSearch(problema)

ini = time() # Tempo inicial

dfs_solucao, dfs_estados_visitados, dfs_num_visitados = dfs.busca(start, target) # chamando busca

dfs_time = time()-ini # Tempo total

if dfs_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Visitando #1
Visitando #2
Visitando #3
Solution found!!!


In [27]:
# Apresentando resultados
print("==== BFS ====")
print(f"Solução encontrada? {bfs_solucao}")
print(f"Número de estados visitados: {bfs_num_visitados}")
print(f"Tempo de execução: {bfs_time}")

print("==== DFS ====")
print(f"Solução encontrada? {dfs_solucao}")
print(f"Número de estados visitados: {dfs_num_visitados}")
print(f"Tempo de execução: {dfs_time}")

==== BFS ====
Solução encontrada? True
Número de estados visitados: 18
Tempo de execução: 0.01629805564880371
==== DFS ====
Solução encontrada? True
Número de estados visitados: 3
Tempo de execução: 0.0028181076049804688


### Exercício

Alterar a matriz inicial de posições para a apresentada na imagem abaixo e avaliar a performance das duas abordagens de busca cega

<img src="images/exercicio.png" width="250" align="center">

In [28]:
# Pacote auxiliar para o cálculo do tempo
from time import time

# Criando objeto do problema
problema = SlidingPuzzle(3)

# Criando Matriz inicial e matriz alvo
start = np.matrix([[2,8,3],[1,6,4],[7,0,5]])
target = np.matrix([[1,2,3],[8,0,4],[7,6,5]])

# Mostrando informações iniciais
print(f"Initial state: \n{start}")
print("*"*15)
print(f"Target state: \n{target}")
print("*"*15)

Initial state: 
[[2 8 3]
 [1 6 4]
 [7 0 5]]
***************
Target state: 
[[1 2 3]
 [8 0 4]
 [7 6 5]]
***************


In [29]:
# Execução do BFS
bfs = BreadthFirstSearch(problema)

ini = time() # Tempo inicial

bfs_solucao, bfs_estados_visitados, bfs_num_visitados = bfs.busca(start, target) # chamando busca

bfs_time = time()-ini # Tempo total

if bfs_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Visitando #1
Visitando #2
Visitando #3
Visitando #4
Visitando #5
Visitando #6
Visitando #7
Visitando #8
Visitando #9
Visitando #10
Visitando #11
Visitando #12
Visitando #13
Visitando #14
Visitando #15
Visitando #16
Visitando #17
Visitando #18
Visitando #19
Visitando #20
Visitando #21
Visitando #22
Visitando #23
Visitando #24
Visitando #25
Visitando #26
Visitando #27
Visitando #28
Visitando #29
Visitando #30
Visitando #31
Visitando #32
Visitando #33
Visitando #34
Solution found!!!


In [30]:
# Execução do DFS
dfs = DepthFirstSearch(problema)

ini = time() # Tempo inicial

dfs_solucao, dfs_estados_visitados, dfs_num_visitados = dfs.busca(start, target) # chamando busca

dfs_time = time()-ini # Tempo total

if dfs_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Visitando #1
Visitando #2
Visitando #3
Visitando #4
Visitando #5
Visitando #6
Visitando #7
Visitando #8
Visitando #9
Visitando #10
Visitando #11
Visitando #12
Visitando #13
Visitando #14
Visitando #15
Visitando #16
Visitando #17
Visitando #18
Visitando #19
Visitando #20
Visitando #21
Visitando #22
Visitando #23
Visitando #24
Visitando #25
Visitando #26
Visitando #27
Visitando #28
Visitando #29
Visitando #30
Visitando #31
Visitando #32
Visitando #33
Visitando #34
Visitando #35
Visitando #36
Visitando #37
Visitando #38
Visitando #39
Visitando #40
Visitando #41
Visitando #42
Visitando #43
Visitando #44
Visitando #45
Visitando #46
Visitando #47
Visitando #48
Visitando #49
Visitando #50
Visitando #51
Visitando #52
Visitando #53
Visitando #54
Visitando #55
Visitando #56
Visitando #57
Visitando #58
Visitando #59
Visitando #60
Visitando #61
Visitando #62
Visitando #63
Visitando #64
Visitando #65
Visitando #66
Visitando #67
Visitando #68
Visitando #69
Visitando #70
Visitando #71
Visitando #72
V

In [31]:
# Apresentando resultados
print("==== BFS ====")
print(f"Solução encontrada? {bfs_solucao}")
print(f"Número de estados visitados: {bfs_num_visitados}")
print(f"Tempo de execução: {bfs_time}")

print("==== DFS ====")
print(f"Solução encontrada? {dfs_solucao}")
print(f"Número de estados visitados: {dfs_num_visitados}")
print(f"Tempo de execução: {dfs_time}")

==== BFS ====
Solução encontrada? True
Número de estados visitados: 34
Tempo de execução: 0.06946516036987305
==== DFS ====
Solução encontrada? False
Número de estados visitados: 2000
Tempo de execução: 21.085301637649536
